In [10]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
from coeditor.dataset import *
from coeditor.encoding import WindowArgs
from spot.utils import pretty_print_dict

os.chdir(proj_root())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# window = WindowArgs(4096)
# encoder = ProjectLevelEditTokenizer(window)
# data = dataset_from_projects([proj_root()], encoder, max_history_per_repo=20)

Create tokenized edits: 100%|██████████| 1/1 [00:10<00:00, 10.57s/chunk]


In [13]:
dataset_name = "medium"
save_dir = get_dataset_dir(dataset_name) / "tokenized-file_based"
datasets = load_datasets(save_dir)

In [14]:
for group, dataset in datasets.items():
    print("=" * 20, group, "=" * 20)
    pretty_print_dict(dataset.overall_stats())

==================== test ====================
n_projects: 20
n_edits: 12666
input_size:
   mean: 2635.5
   median: 2735
   min: 40
   max: 4096
output_size:
   mean: 113.73
   median: 75
   min: 3
   max: 4776
==================== valid ====================
n_projects: 10
n_edits: 2297
input_size:
   mean: 2733.4
   median: 2816
   min: 54
   max: 4096
output_size:
   mean: 116.93
   median: 89
   min: 5
   max: 1567
==================== train ====================
n_projects: 100
n_edits: 50804
input_size:
   mean: 2485.5
   median: 2508
   min: 33
   max: 4096
output_size:
   mean: 98.524
   median: 64
   min: 3
   max: 8434


In [ ]:
for group in datasets:
    if not (save_dir / f"{group}.pkl").exists():
        continue
    dataset = pickle_load(save_dir / f"{group}.pkl")
    print("=" * 20, group, "=" * 20)
    display(dataset.per_repo_stats())

In [ ]:
print(list(datasets["test"].all_edits())[1].show_prediction())